https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb?ref=blog.langchain.dev

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
search_key = os.environ["AZURE_SEARCH_ADMIN_KEY"]

In [2]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

from langchain.vectorstores.azuresearch import AzureSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)

import re

## [00] define a state dict to contain - task, plan, steps, and other variables.

In [3]:
from typing import TypedDict, List


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

## [0] Embedding/vectorisation

In [4]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    chunk_size=1 
)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=search_endpoint,
    azure_search_key=search_key,
    index_name="boardai03",
    embedding_function=embeddings.embed_query,
)

[llm]

In [5]:
llm = AzureChatOpenAI(
    deployment_name="gpt-4",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    # temperature=0
)

In [6]:
retriever = vector_store.as_retriever(search_key="hybrid", search_kwargs={"k": 2})

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# model = (
# {"context": retriever | format_docs, "question": RunnablePassthrough()}
# | prompt
# | llm
# | StrOutputParser()
# )

In [7]:
# while True:
#     query = input("User: ")
#     if query == "exit":
#         break
#     print("query:",query , "\nanswer: ", model.invoke(query))

## [1] Planner
https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb?ref=blog.langchain.dev

In [8]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:

(1) RAG+LLM[input]: A pretrained RAG+LLM like yourself. Useful when you need to act with general
world knowledge and common sense + specific knowledge and data especially for Board and secretary role. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

(2) sendEmail_chase_gatherPaper[input]: Worker that send emails to board members to ask write board paper, check status of the return of those email and chase. 
Input will be a list of board members and agendas.

(3) compilePaper[input]:Worker that add all the separate paper into one board paper. input will be multiple board paper pieces from board members.

(4) circulatePaper[input]: Worker that send compiled board paper to board members and get feed back and fix. Input will be a board paper. 

(5) sendReminderEmail[input]: xyz

For example,
Task: execute this: To create a board paper, first, based on an approved agenda, the secretary sends an email to the NEC (or board members). The NEC then writes the paper. 
After the paper is written, the secretary compile, circulates and distributes the paper to the board members.


Plan: Find out what is approved agenda. #E1 = RAG+LLM[what is approved agenda]

Plan: Find out who are the NEC or board members. #E2 = RAG+LLM[who are the NEC or board members]

Plan: the secretary sends an email to the NEC (or board members).The NEC writes the paper. #E3 = sendEmail_chase_gatherPaper[#E2]

Plan: compilePaper #E4 = compilePaper[#E3] 

Plan: After the paper is written, the secretary circulates and distributes the paper to the board members. #E5 = circulatePaper[#E4]


Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.
\nQuestion: {question}
\nContext: {context}
\nAnswer:""" 


In [9]:
# rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt

In [10]:
# simply Converting prompt into p_prompt: ChatPromptTemplate as langchain receives in that form

# p_prompt = ChatPromptTemplate(input_variables=['context', 'question'], 
#                             metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, 
#                             messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template=prompt))])


# same result as code above
p_prompt = ChatPromptTemplate.from_messages([("user", prompt)])

In [11]:
model = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| p_prompt
| llm
| StrOutputParser()
)

In [12]:
query = "how do I create a board paper?"
res = model.invoke(query)


In [13]:
print(res)

Plan: Understand the approved agenda for the board paper. #E1 = RAG+LLM[what is the approved agenda for the board paper]

Plan: Identify the NEC or board members who will contribute to the paper. #E2 = RAG+LLM[who are the NEC or board members]

Plan: The approved agenda and the list of NEC or board members will be sent in an email asking them to contribute to the board paper. #E3 = sendEmail_chase_gatherPaper[#E1, #E2]

Plan: After all contributions have been received, compile them into one comprehensive board paper. #E4 = compilePaper[#E3]

Plan: Once the board paper is compiled, circulate it among the board members for their review and feedback. Make any necessary revisions based on the feedback received. #E5 = circulatePaper[#E4]

Plan: If there's a lack of response or feedback, send a reminder email to ensure all board members are engaged and informed. #E6 = sendReminderEmail[#E2] 

Plan: Distribute the final board paper to all board members for their use during the board meeting. 

Planner Node

To connect the planner to our graph,
we will create a "get_plan" node 
- accepts the ReWOO state and 
- returns with a state update for the steps and plan_string fields.

In [14]:
# Regex to match expressions of the form  "Plan: E#... = ...[...]"
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]" 

# prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
# planner = prompt_template | model


# class ReWOO(TypedDict):
#     task: str
#     plan_string: str
#     steps: List
#     results: dict
#     result: str


def get_plan(state: ReWOO):
    task = state["task"]
    result = model.invoke({"task": task}) #ask agent about the each task/plan again 
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content) # matches = end nodes with #E result/task
    
    return {"steps": matches, "plan_string": result.content}
    
    # plan_string = Plan e.g. Plan: ..... , Plan: #E[] = ....
    # steps = only this part e.g. Plan: #E[] = ....


## [2] EXECUTOR

Define the tool execution node

In [15]:
# load plugins & test

from plugins.basic import BasicPlugins

plugin_instance = BasicPlugins()

result = plugin_instance.email_function()
result = plugin_instance.compile_paper()
result = plugin_instance.circulate_paper()


result

'circulate paper'

In [16]:
def _get_current_task(state: ReWOO):
    if state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1
    

# initialise plugins
from plugins.basic import BasicPlugins
plugin_instance = BasicPlugins()


# do i need to mention all the tools here? semantic kernel doesn't need this stage...hrmmmmm try langchain ver first for now
    

def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1] # state["steps"] = List   # how come this is a tuple like structure
    _results = state["results"] or {}

    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    

    if tool == "RAG_LLM":
        result = model.invoke(tool_input)
    elif tool =="EMAIL":
        result = plugin_instance.email_function()
    elif tool =="COMPILE":
        result = plugin_instance.compile_paper()
    elif tool =="CIRCULATE":
        result = plugin_instance.circulate_paper()
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}

## [3] Solver 

The solver receives the full plan and generates the final response based on the responses of the tool calls from the worker.

Solver has its own prompt.

In [17]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""

def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = state["results"] or {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)                                      # should I use LLM or RAG + LLM ???? Try just LLM first, will be faster
    return {"result": result.content}

## [4] Define Graph

Our graph defines the workflow. 

Each of the planner, worker(tool executor), and solver modules are added as nodes.

In [18]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [19]:
from langgraph.graph import StateGraph, END

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.set_entry_point("plan")

app = graph.compile()

In [20]:
task = "how do I create a board paper?"

In [21]:


for s in app.stream({"task": task}):
    print(s)
    print("---")

TypeError: expected string or buffer